# Data Processing

The first step in data processing is obtaining and preparing the data. We used the Animal Sound Archive, it is available in the folder `animal-sound`.

As one can see, there are many files in that folder. The relevants files are `occurrence.txt` and `multimedia.txt`. The first has information about each ocurrence, and the second has the links to the audio files.

## Imports

In [1]:
import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get
from tqdm.notebook import tqdm

import os
import warnings

import requests

import torchaudio
import torchaudio.transforms as T

import matplotlib.pyplot as plt

## Merging dataframes

Before anything else, we have to merge the relevant data in the files.

In [2]:
ocurrences = pd.read_csv("../animal-sound/occurrence.txt",
                         delimiter="\t")
ocurrences.head()

/home/snoooze/Git/Animal-Sound/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (198) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,gbifID,abstract,accessRights,accrualMethod,accrualPeriodicity,accrualPolicy,alternative,audience,available,bibliographicCitation,...,identifiedByID,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
0,1572324720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DEU,Germany,DEU.4_1,Brandenburg,DEU.4.18_1,Uckermark,DEU.4.18.11_1,Schwedt/Oder,LC
1,1572324719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DEU,Germany,DEU.4_1,Brandenburg,DEU.4.18_1,Uckermark,DEU.4.18.11_1,Schwedt/Oder,LC
2,1572324718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DEU,Germany,DEU.4_1,Brandenburg,DEU.4.18_1,Uckermark,DEU.4.18.11_1,Schwedt/Oder,LC
3,1572324717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DEU,Germany,DEU.4_1,Brandenburg,DEU.4.18_1,Uckermark,DEU.4.18.1_1,Angermünde,LC
4,1572324716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DEU,Germany,DEU.4_1,Brandenburg,DEU.4.18_1,Uckermark,DEU.4.18.11_1,Schwedt/Oder,LC


We can see that the `ocurrences.txt` file has a lot of columns, and most of them are filled with `NaN`. We have to get only the columns that are relevant to the problem: 
 - **gbifID**
 - **species**
 - **genus**
 - **family**
 - **class**
 - **phylum**

In [3]:
ocurrences = ocurrences[["gbifID", "species", "genus", "family", "class", "phylum"]]
ocurrences.head()

,gbifID,species,genus,family,class,phylum
0,1572324720,Crex crex,Crex,Rallidae,Aves,Chordata
1,1572324719,Crex crex,Crex,Rallidae,Aves,Chordata
2,1572324718,Crex crex,Crex,Rallidae,Aves,Chordata
3,1572324717,Crex crex,Crex,Rallidae,Aves,Chordata
4,1572324716,Crex crex,Crex,Rallidae,Aves,Chordata


In [4]:
multimedia = pd.read_csv("../animal-sound/multimedia.txt",
                         delimiter="\t")
multimedia.head()

,gbifID,type,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder
0,1572324720,Sound,audio/mpeg,http://www.tierstimmenarchiv.de/recordings/Cre...,http://www.tierstimmenarchiv.de/webinterface/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,NaN
1,1572324719,Sound,audio/mpeg,http://www.tierstimmenarchiv.de/recordings/Cre...,http://www.tierstimmenarchiv.de/webinterface/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,NaN
2,1572324718,Sound,audio/mpeg,http://www.tierstimmenarchiv.de/recordings/Cre...,http://www.tierstimmenarchiv.de/webinterface/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,NaN
3,1572324717,Sound,audio/mpeg,http://www.tierstimmenarchiv.de/recordings/Cre...,http://www.tierstimmenarchiv.de/webinterface/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,NaN
4,1572324716,Sound,audio/mpeg,http://www.tierstimmenarchiv.de/recordings/Cre...,http://www.tierstimmenarchiv.de/webinterface/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://creativecommons.org/licenses/by-nc-sa/4.0/,NaN


The `multimedia.txt` file has irrelevant columns. We only need the **gbifID** and **identifier** columns. 

In [5]:
multimedia = multimedia[["gbifID", "identifier"]]
multimedia.head()

,gbifID,identifier
0,1572324720,http://www.tierstimmenarchiv.de/recordings/Cre...
1,1572324719,http://www.tierstimmenarchiv.de/recordings/Cre...
2,1572324718,http://www.tierstimmenarchiv.de/recordings/Cre...
3,1572324717,http://www.tierstimmenarchiv.de/recordings/Cre...
4,1572324716,http://www.tierstimmenarchiv.de/recordings/Cre...


Now we have to combine the 2 datasets merging them by the **gbifID**.

In [6]:
df = multimedia.merge(ocurrences, on="gbifID", how="inner")
df.head()

,gbifID,identifier,species,genus,family,class,phylum
0,1572324720,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata
1,1572324719,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata
2,1572324718,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata
3,1572324717,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata
4,1572324716,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata


In [7]:
df.shape

(16385, 7)

The next step is to remove the samples with `NaN` in any column.

In [8]:
df = df.dropna()
df.shape

(16270, 7)

## Download the audios

Now that we have all the links of the audios, we have to save them.

In [9]:
tqdm.pandas(desc="Downloading files")

In [10]:
def createAudioFile(row):
  ocurr_id = str(row["gbifID"])
  phylum = row["phylum"]
  class_name = row["class"]
  family = row["family"]
  genus = row["genus"]
  species = row["species"]
  url = row["identifier"]
  
  folder = '{}/{}/{}/{}/{}/'.format(phylum, class_name, family, genus, species).replace(" ", "_")
  
  file_name = folder + ocurr_id + ".mp3"
  folder_name = r"../data/{}".format(folder)

  if not os.path.exists('../data/' + folder):
    os.makedirs(folder_name, exist_ok=True)

  path = r"../data/{}".format(file_name)
  
  if not os.path.exists(path):
    with open(path, "wb") as f:
      f.write(requests.get(url).content)
    
  return file_name

a = df.sample(2, random_state=1).apply(createAudioFile, axis=1)

In [11]:
# Creating dask dataframe
ddf = dd.from_pandas(df, npartitions=16)

# df["file_name"] = df.progress_apply(createAudioFile, axis=1)
res = ddf.map_partitions(lambda df: df.apply((lambda row: createAudioFile(row)), axis=1)).compute(scheduler=get)


We have to remove corrupted data

In [12]:
tqdm.pandas(desc="Dropping corrupted files")

In [13]:
def isCorrupted(row):
  try:
    with warnings.catch_warnings():
      warnings.simplefilter("ignore")
      torchaudio.load("../data/" + row.file_name)
      return False
  except:
    return True

In [14]:
df["is_corrupted"] = df.progress_apply(isCorrupted, axis=1)

Dropping corrupted files:   0%|          | 0/16270 [00:00<?, ?it/s]

In [15]:
df.head(10)

,gbifID,identifier,species,genus,family,class,phylum,is_corrupted
0,1572324720,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,True
1,1572324719,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,True
2,1572324718,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,True
3,1572324717,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,True
4,1572324716,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,True
5,1572324715,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,True
6,1572324714,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,True
7,1571202165,http://www.tierstimmenarchiv.de/recordings/Phy...,Phylloscopus ibericus,Phylloscopus,Phylloscopidae,Aves,Chordata,True
8,1571067681,http://www.tierstimmenarchiv.de/recordings/Mül...,Mulleripicus pulverulentus,Mulleripicus,Picidae,Aves,Chordata,True
9,1571067680,http://www.tierstimmenarchiv.de/recordings/Mül...,Mulleripicus pulverulentus,Mulleripicus,Picidae,Aves,Chordata,True


In [16]:
df = df.drop(df[df.is_corrupted].index).drop(columns="is_corrupted")

In [17]:
df.shape

(0, 7)

Finaly, we save the file

In [18]:
df = df.reset_index(drop=True)

In [19]:
df.to_csv("../datasets/AnimalSoundFull.csv", index=False)